# Step 1 & 2

In [ ]:
import cv2
import numpy as np

# Load a checkerboard image
img = cv2.imread('checkerboard.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Define chessboard size (columns, rows)
checkerboard_size = (6, 9)

# Find chessboard corners
ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

# Draw detected corners
if ret:
    img = cv2.drawChessboardCorners(img, checkerboard_size, corners, ret)
    cv2.imshow('Detected Corners', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Step 3 & 4

In [ ]:
import cv2
import numpy as np
import glob
import os

# Define the chessboard size (columns, rows)
checkerboard_size = (6, 9)

# Prepare object points
objp = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)

# Store object points and image points
objpoints = []  # Real world 3D points
imgpoints = []  # 2D points in image plane

# Define the path to the folder containing calibration images
image_folder = r'C:\Users\m.nasif\Desktop\CV\Lab 5\Images'


# Get all image file paths in the folder
image_files = glob.glob(os.path.join(image_folder, '*.jpg'))  # Change '*.jpg' to '*.png' if needed
# Process each image
for fname in image_files:
    img = cv2.imread(fname)
    
    if img is None:
        print(f"Warning: Could not read {fname}")
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)


# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
# Print the camera matrix and distortion coefficients
print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist)


Camera Matrix:
 [[768.01528126   0.         323.73970904]
 [  0.         771.20067921 271.64518196]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[-4.68051416e-02 -1.08256329e+00  4.13575440e-03  1.73394826e-03
   7.77707281e+00]]


# Step 5

In [24]:
import cv2
import numpy as np

# Load the camera calibration parameters
# Ensure 'mtx' and 'dist' are correctly obtained from the calibration step
if 'mtx' not in globals() or 'dist' not in globals():
    print("Error: Camera calibration parameters (mtx, dist) not found. Run calibration first.")
    exit()

# Load an image
img = cv2.imread(r'C:\Users\m.nasif\Desktop\CV\Lab 5\Images\0.jpg')

# Check if the image is loaded properly
if img is None:
    print("Error: Could not read the image file. Please check the path.")
    exit()

# Get image size
h, w = img.shape[:2]

# **Step 1: Apply Artificial Distortion**
def apply_distortion(img, mtx, dist):
    distorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    return distorted_img

distorted_img = apply_distortion(img, mtx, dist)

# **Step 2: Remove the Distortion**
# Compute the optimal new camera matrix
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort the image
undistorted_img = cv2.undistort(distorted_img, mtx, dist, None, new_camera_mtx)

# Crop the image based on the region of interest (ROI)
x, y, w, h = roi
undistorted_img = undistorted_img[y:y+h, x:x+w]

# **Display results**
cv2.imshow('Original Image', img)
cv2.imshow('Distorted Image', distorted_img)
cv2.imshow('Undistorted Image', undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Bonus

In [17]:
import cv2
import numpy as np

# Ensure calibration parameters exist
if 'mtx' not in globals() or 'dist' not in globals():
    print("Error: Camera calibration parameters not found. Run calibration first.")
    exit()

# Load image
img = cv2.imread(r'C:\Users\m.nasif\Desktop\CV\Lab 5\Images\0.jpg')
if img is None:
    print("Error: Could not read the image file.")
    exit()

h, w = img.shape[:2]

# Print camera matrix and distortion coefficients
print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist)

# Compute an optimized camera matrix
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), alpha=0.6, newImgSize=(w, h))

# Apply undistortion
undistorted_img = cv2.undistort(img, mtx, dist, None, new_camera_mtx)

# Crop the final undistorted image
x, y, w, h = roi
undistorted_img = undistorted_img[y:y+h, x:x+w]

# Display images
cv2.imshow('Original Image', img)
cv2.imshow('Undistorted Image', undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Camera Matrix:
 [[768.01528126   0.         323.73970904]
 [  0.         771.20067921 271.64518196]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[-4.68051416e-02 -1.08256329e+00  4.13575440e-03  1.73394826e-03
   7.77707281e+00]]


In [20]:
import cv2
import numpy as np

# Define the chessboard size (columns, rows)
checkerboard_size = (6, 9)

# 3D coordinates for a cube centered at each chessboard square
cube_size = 1  # Adjust the size of the cubes
cube_points = np.float32([
    [0, 0, 0], [cube_size, 0, 0], [cube_size, cube_size, 0], [0, cube_size, 0], 
    [0, 0, -cube_size], [cube_size, 0, -cube_size], [cube_size, cube_size, -cube_size], [0, cube_size, -cube_size]
])

# Function to draw a cube at the given image points
def draw_cube(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)

    # Draw base
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), 2)

    # Draw pillars
    for i, j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255, 0, 0), 2)

    # Draw top
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 2)

    return img

# Capture video from webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret:
        # Refine detected corners
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), 
                                   (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001))

        # Solve the pose of the chessboard
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners, mtx, dist)

        # Loop over multiple squares and place cubes
        for i in range(0, len(corners), 5):  # Place a cube every 5th square
            square_origin = np.float32([objp[i]])  # Select one chessboard square as origin
            cube_3d = cube_points + square_origin  # Move the cube to that square
            imgpts, _ = cv2.projectPoints(cube_3d, rvecs, tvecs, mtx, dist)
            frame = draw_cube(frame, corners, imgpts)

    # Display the AR video
    cv2.imshow('Augmented Reality Chessboard', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
